<a href="https://colab.research.google.com/github/idc315/presentation-1-group-9/blob/main/Box_office_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df_2=pd.read_csv("train.csv")
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [ ]:
df_2_new=df_2.drop(["id","belongs_to_collection","genres","imdb_id","original_language","original_title","overview","popularity","poster_path","production_companies","production_countries","release_date","spoken_languages","status","tagline","Keywords","cast","crew"],axis=1)
df_2_new

,budget,homepage,runtime,title,revenue
0,14000000,NaN,93.0,Hot Tub Time Machine 2,12314651
1,40000000,NaN,113.0,The Princess Diaries 2: Royal Engagement,95149435
2,3300000,http://sonyclassics.com/whiplash/,105.0,Whiplash,13092000
3,1200000,http://kahaanithefilm.com/,122.0,Kahaani,16000000
4,0,NaN,118.0,Marine Boy,3923970
...,...,...,...,...,...
2995,0,NaN,102.0,Chasers,1596687
2996,0,NaN,102.0,We Are the Best!,180590
2997,65000000,NaN,120.0,The Long Kiss Goodnight,89456761
2998,42000000,http://www.alongcamepolly.com/,90.0,Along Came Polly,171963386


In [ ]:
df_2_new.budget.corr(df_2_new.revenue)

0.7529645103815286

As correlation efficiency is great than 0.7, it suggest that budget and revenue have a very close relationship. With this highly positive correlation coefficient, I expect a higher budget film will have higher revenue.

In [ ]:
runtime_median = df_2_new['runtime'].median()

long_movies = df_2_new[df_2_new['runtime'] > runtime_median]['revenue']
short_movies = df_2_new[df_2_new['runtime'] <= runtime_median]['revenue']

from scipy import stats

t_statistic, p_value = stats.ttest_ind(long_movies, short_movies,alternative='greater')

print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

# Null hypothesis: There is no relation between runtime and revenue
# Alternative hypothesis: Longer movies have higher revenue
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis.")
    print("There is evidence to suggest that longer movies have higher revenue.")
else:
    print("Fail to reject the null hypothesis.")
    print("There is not enough evidence to suggest that longer movies have higher revenue.")

        budget                              homepage  runtime  \
1     40000000                                   NaN    113.0   
2      3300000     http://sonyclassics.com/whiplash/    105.0   
3      1200000            http://kahaanithefilm.com/    122.0   
4            0                                   NaN    118.0   
10     1000000                                   NaN    119.0   
...        ...                                   ...      ...   
2992   1135654                                   NaN    149.0   
2993  60000000  http://www.theterminal-themovie.com/    128.0   
2994         0                                   NaN    106.0   
2997  65000000                                   NaN    120.0   
2999  35000000      http://www.abductionthefilm.com/    106.0   

                                         title    revenue  
1     The Princess Diaries 2: Royal Engagement   95149435  
2                                     Whiplash   13092000  
3                                      

- P-values here assume the null hypothesis to be correct
and serve to determine the likelihood of obtaining a
test result by chance.  
- If the p-value was found to be 0.02, there is evidence to suggest that longer movies have higher revenue a significance level of 0.05 .

In [ ]:
df_homepage = df_2_new.homepage.notnull().astype(int)
df_homepage.sum()
max_revenue=df_2_new.loc[df_2_new['revenue'].idxmax(), 'title']

print(f"There are {df_homepage.sum()} movies with a homepage.")
print(f"The median runtime of movies in the dataset is: {runtime_median} minutes.")
print(f"The movie in the dataset has the highest revenue is : {max_revenue} ")

There are 946 movies with a homepage.
The median runtime of movies in the dataset is: 104.0 minutes.
The movie in the dataset has the highest revenue is : The Avengers 


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

X = df_2_new[['budget', 'runtime']]
y = df_2_new['revenue']


imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.567118913194349

With the R squared score is 0.57, the model is fit mediumly with the test



In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

X = df_2_new[['budget', 'runtime']]
y = df_2_new['revenue']

imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.09381297188883142